<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [59]:
!pip install -q datasets pandas pillow tqdm huggingface_hub decord

In [60]:
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
from huggingface_hub import login
import getpass

In [61]:
# Authenticate Hugging Face
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(HF_TOKEN)

Enter your Hugging Face token: ··········


In [62]:
# Set download directory
BASE_DIR = Path.home() / "Downloads" / "deepfake_dataset"
BASE_DIR.mkdir(parents=True, exist_ok=True)
metadata = []

In [63]:
def download_data(dataset_name, category_name, file_ext, key):
    """Downloads and saves data (images, videos, audio)"""
    print(f"\n📥 Downloading {category_name} Data...")
    try:
        dataset = load_dataset(dataset_name, split="train")
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {e}")
        return

    save_dir = BASE_DIR / category_name
    save_dir.mkdir(parents=True, exist_ok=True)

    for category, label in [("real", 0), ("fake", 1)]:
        (save_dir / category).mkdir(parents=True, exist_ok=True)
        samples = dataset.filter(lambda x: x["label"] == label).select(range(20))

        for i, sample in enumerate(tqdm(samples, desc=f"Saving {category} {category_name}")):
            file_path = save_dir / category / f"{category}_{i:03d}.{file_ext}"
            with open(file_path, "wb") as f:
                f.write(sample[key]["bytes"] if isinstance(sample[key], dict) else sample[key])

            metadata.append({
                "modality": category_name,
                "category": category,
                "filename": file_path.name,
                "file_path": str(file_path),
                "source_dataset": dataset_name
            })

def save_metadata():
    """Saves metadata as CSV"""
    pd.DataFrame(metadata).to_csv(BASE_DIR / "metadata.csv", index=False)
    print("✅ Metadata saved")

def main():
    download_data("thenewsupercell/fixed-fakeavceleb", "audio", "wav", "audio")
    download_data("mkhLlamaLearn/dfdc", "videos", "mp4", "video")
    download_data("taohu/faceforensics_h5", "images", "jpg", "image")
    save_metadata()
    print(f"\n🎉 Dataset Collection Complete! Data stored in {BASE_DIR}")

In [64]:
if __name__ == "__main__":
    main()


📥 Downloading audio Data...


Filter:   0%|          | 0/15999 [00:00<?, ? examples/s]

ImportError: To support decoding videos, please install 'decord'.